In [ ]:
%%capture
!pip install transformers[torch]
!pip install datasets transformers sentencepiece

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

In [ ]:
test = pd.read_csv("test_ready_for_models.csv", usecols=['text_mod', 'is_sarcastic'])
train = pd.read_csv("train_ready_for_models.csv", usecols=['text_mod', 'is_sarcastic'])

In [ ]:
train

,is_sarcastic,text_mod
0,1,поїхали на**й
1,0,ноги колготок роблять в індустріальний кругові...
2,1,"ну так, чесно працювати -- то для лохів"
3,1,яка нсподіванка
4,0,"тема норм, але вбіса багато писати -_-../"
...,...,...
8599,0,"бля, дєд, та досить вже ламати твітор."
8600,1,у мене зп в доларах😂
8601,0,"нє, то якось слєгка слішком. голі щиколотки пі..."
8602,1,"поки мене нема вже 2-й тиждень на парах, група..."


In [ ]:
test

,is_sarcastic,text_mod
0,1,"ооо, ми розбили їм вікно! оце достойна відпові..."
1,1,"""називайте мене богом""\nти шо ягамі лайт? 🙄🙄🙄💅"
2,1,може ще ставки по кредитах знизяться?..
3,1,заїбали ці дойобувачі з своєю принциповістю 🙄🙄...
4,1,"""так""? ""так""????? я теж хочу, де мої фанфікі ..."
...,...,...
2147,0,"а ще чувіху з школи, в якій вчиться моя подруг..."
2148,0,і нє ввєді мєня воіскушеніє
2149,1,ура тривожність
2150,1,"всмислі ви не використовуєте, а шо можна чимо..."


In [ ]:
test.rename(columns={'is_sarcastic':'label'}, inplace=True)
train.rename(columns={'is_sarcastic':'label'}, inplace=True)

In [ ]:
test = Dataset.from_pandas(test)
test = test.class_encode_column("label")
dev_test_dataset = test.train_test_split(test_size=0.5, stratify_by_column="label")

Stringifying the column:   0%|          | 0/2152 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2152 [00:00<?, ? examples/s]

In [ ]:
train = Dataset.from_pandas(train)
train = train.class_encode_column("label")

Stringifying the column:   0%|          | 0/8604 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/8604 [00:00<?, ? examples/s]

In [ ]:
train_dev_test = DatasetDict({
    "train": train,
    "dev":dev_test_dataset["train"],
    "test": dev_test_dataset["test"]
})

In [ ]:
train_dev_test

DatasetDict({
    train: Dataset({
        features: ['label', 'text_mod'],
        num_rows: 8604
    })
    dev: Dataset({
        features: ['label', 'text_mod'],
        num_rows: 1076
    })
    test: Dataset({
        features: ['label', 'text_mod'],
        num_rows: 1076
    })
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_ckpt = "youscan/ukr-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
tokenizer.vocab_size

52000

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>'}

In [ ]:
train.iloc[2]

is_sarcastic                                             1
text_mod          ну так, чесно працювати -- то для лохів 
Name: 2, dtype: object

In [ ]:
encoded = tokenizer(train['text_mod'][2])

In [ ]:
for token in encoded['input_ids']:
    print(token, tokenizer.decode(token))

0 <s>
225  
6148  ну
515  так
16 ,
13615  чесно
3325  працювати
6932  --
763  то
469  для
385  л
946 ох
317 ів
225  
2 </s>


In [ ]:
def tokenize_texts(examples):
    return tokenizer(examples["text_mod"], padding=True, truncation=True, max_length=514)

In [ ]:
tokenized_train_dev_test = train_dev_test.map(tokenize_texts, batched=True)
tokenized_train_dev_test

Map:   0%|          | 0/8604 [00:00<?, ? examples/s]

Map:   0%|          | 0/1076 [00:00<?, ? examples/s]

Map:   0%|          | 0/1076 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text_mod', 'input_ids', 'attention_mask'],
        num_rows: 8604
    })
    dev: Dataset({
        features: ['label', 'text_mod', 'input_ids', 'attention_mask'],
        num_rows: 1076
    })
    test: Dataset({
        features: ['label', 'text_mod', 'input_ids', 'attention_mask'],
        num_rows: 1076
    })
})

In [ ]:
id2label = {1: "sarcastic",0: "not_sarcastic"}
label2id = {"sarcastic": 1, "not_sarcastic": 0}

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=2, label2id=label2id, id2label=id2label).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at youscan/ukr-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
predicted_labels = []
ground_truth_labels = []

# Iterate through the test data and make predictions on unfine-tuned model
for example in tokenized_train_dev_test['test']:
    text_input = example["text_mod"]

    tokenized_input = tokenizer(text_input, return_tensors="pt")

    tokenized_input = {key: value.to("cuda:0") for key, value in tokenized_input.items()}

    predictions = model(**tokenized_input)

    predicted_label = predictions.logits.argmax().item()

    predicted_labels.append(predicted_label)

    if "label" in example:
        ground_truth_labels.append(example["label"])

In [ ]:
%%capture
!pip install evaluate

In [ ]:
from evaluate import load

metrics = {
    "accuracy": load("accuracy"),
    "precision": load("precision"),
    "recall": load("recall"),
    "f1": load("f1"),
}


metric_results = {}

for metric_name, metric in metrics.items():
    result = metric.compute(predictions=predicted_labels, references=ground_truth_labels)
    metric_results[metric_name.capitalize()] = result

for metric_name, result in metric_results.items():
    print(f"{metric_name}: {result}")

Accuracy: {'accuracy': 0.4944237918215613}
Precision: {'precision': 0.3333333333333333}
Recall: {'recall': 0.011152416356877323}
F1: {'f1': 0.02158273381294964}


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    roc_auc = roc_auc_score(labels, predictions)  # Only applicable if you have probabilities

    return {"accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "roc_auc": roc_auc}

In [1]:
from transformers import TrainingArguments

batch_size = 16
num_train_epochs = 3
logging_steps = len(tokenized_train_dev_test["train"]) // (batch_size * num_train_epochs)

args = TrainingArguments(
    output_dir=f"{model_ckpt}-finetuned-sarc",
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.02,
    logging_steps=logging_steps,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_dev_test["train"],
    eval_dataset=tokenized_train_dev_test["dev"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_train_dev_test['test'])

{'eval_loss': 0.45003125071525574,
 'eval_accuracy': 0.7732342007434945,
 'eval_precision': 0.8868421052631579,
 'eval_recall': 0.6263940520446096,
 'eval_f1': 0.7342047930283224,
 'eval_roc_auc': 0.7732342007434944,
 'eval_runtime': 6.4292,
 'eval_samples_per_second': 167.362,
 'eval_steps_per_second': 10.577,
 'epoch': 3.0}